In [8]:
from __future__ import print_function
import datetime
import numpy as np
import pandas as pd
import akshare as ak
import pandas_datareader.data as web
# 计算年化收益率
def annualised_sharpe(returns, N=252):
    return np.sqrt(N) * returns.mean() / returns.std()

In [21]:
def equity_sharpe(symbol, startdate, enddata):
    df = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=startdate, end_date=enddata, adjust="")

    # Use the percentage change method to easily calculate daily returns
    df['daily_ret'] = df['收盘'].pct_change()
    # Assume an average annual risk-free rate over the period of 3%
    df['excess_daily_ret'] = df['daily_ret'] - 0.03/252
    # Return the annualised Sharpe ratio based on the excess daily returns
    return annualised_sharpe(df['excess_daily_ret'])
equity_sharpe('600000','20230101','20240101')

-0.7304499901503129

这里可以计算出相对%4收益的夏普比，值是负数，小于假设的无风险利率，说明买这个股票还不如存定期3个点。

In [22]:
def market_neutral_sharpe(symbol, index_benchmark,startdate, enddata):
    df_symbol = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=startdate, end_date=enddata, adjust="")
    df_index = ak.index_zh_a_hist(symbol=index_benchmark, period="daily", start_date=startdate, end_date=enddata)
    
    # Calculate the percentage returns on each of the time series
    df_symbol['daily_ret'] = df_symbol['收盘'].pct_change()
    df_symbol.set_index(pd.to_datetime(df_symbol["日期"],format='%Y-%m-%d'),inplace=True)
    df_index['daily_ret'] = df_index['收盘'].pct_change()
    df_index.set_index(pd.to_datetime(df_index["日期"],format='%Y-%m-%d'),inplace=True)
    # 
    strat = pd.DataFrame(index=df_index.index)
    strat['net_ret'] = (df_symbol['daily_ret'] - df_index['daily_ret'])
    # Return the annualised Sharpe ratio for this strategy
    return annualised_sharpe(strat['net_ret'])

market_neutral_sharpe('600000','000001','20230101','20240101')

-0.3123001022491444

相对于和固定三个点相比，把上证指数作为基准，sharp 反而值大些，说明上证指数和固定三个点的定期相比，也不值得考虑，起码从这段历史数据上看是这样。